<h1><center>Anomalies Detection Workflow with LightGBM Framework, Google AI Platfrom and Dataproc</center></h1>
<a id="tc"></a>

## Table of Contents
1. [Configuration](#configuration) 
2. [Prepare TRAIN, VAL, TEST data for LightGBM Model](#preparation)
3. [Train LightGBM Model](#train)<br/>
    3.1. [Train With Default Parameters](#train_with_default)<br/>
    3.2. [Hyperparameters Tuning](#tuning)<br/>
4. [Batch Prediction with TEST Dataset](#prediction)
5. [Anomalies Detection with TEST Dataset. Publish to Grafana](#detection)
6. [Grafana Anomalies Dashboard](#grafana)
7. [Analytics](#analytics)
8. [Push Notebook to GCS Bucket](#gcs)

<a id="configuration"></a>
## Configuration
[back to Table Of Contents](#tc)

In [5]:
def get_transition(transition_file):
    with open(transition_file, 'r') as f:
        return json.load(f)

In [4]:
PROJECT = 'kohls-kos-cicd'
BUCKET = 'ai4ops-main-storage-bucket'
AI_PLATFORM_REGION = 'us-central1'
DATA_BASE_PATH = "apigee_history/apigee/metrics/history"
REGION='global'
SCRIPT_PATH = 'poc/spark/ingest'
AI4OPS_HISTORY_PATH=f"gs://{BUCKET}/apigee_history/apigee/metrics/history"
PROJECT_BASE_PATH='poc'
INPUT_DATA_SOURCE_PATH = ('{}/no_duplicates/ai4ops_remove_duplicates_1559167473,\
                          {}/no_duplicates/ai4ops_remove_duplicates_1559168049,\
                          {}/no_duplicates/ai4ops_remove_duplicates_1559168353'
                          .format(AI4OPS_HISTORY_PATH, 
                                  AI4OPS_HISTORY_PATH, 
                                  AI4OPS_HISTORY_PATH))

DATA_CONFIG = 'lgbm_signature_march_april_may_traffic_not_200.json'


TRAIN_JOB_SUFFIX = 'traffic_not_200_march_april_may'
TUNING_CONFIG_FILE = 'hptuning_config_march_traffic_not_200.yaml' 
AI_PALTFORM_MODEL_NAME = 'api_ai4ops_lgbm_traffic_not_200_march_april_may'

AI_PLATFORM_TRAIN_EXCLUDED_COLUMNS = 'time,metric_val,metric'
AI_PLATFORM_PREDICT_EXCLUDED_INPUT_COLUMNS = '{},var1(t)'.format(AI_PLATFORM_TRAIN_EXCLUDED_COLUMNS)
AI_PLATFORM_PREDICTED_COLUMN_NAME = 'predicted'
AI_PLATFORM_PREDICT_OUTPUT_COLUMNS_MAPPING = 'metric_val=metric,time,var1(t)=value'

AI_PLATFORM_MODEL_BASE_VERSION = 'v1'
CLUSTER = 'ai4ops'


<a id="preparation"></a>
## Prepare TRAIN, VAL, TEST data for LightGBM Model 
See Dataproc Jobs: https://console.cloud.google.com/dataproc/jobs
<br/>[back to Table Of Contents](#tc)


In [ ]:
OUTPUT_PATH = '{}/lgbm-signature'.format(DATA_BASE_PATH)
USE_POWER_TRANSFORMER = False
USE_INVERSE_TRANSFORM = False


In [ ]:
from job_api import *
import importlib
from datetime import datetime
import sys
import pyspark

In [ ]:
builder = DataprocJobBuilder()
session = Session(BUCKET, REGION, CLUSTER, PROJECT)

In [ ]:
SUFFIX = 'traffic_not_200'
TIMESTAMP=int(datetime.now().timestamp())
sign_job_name = "ai4ops_lgbm_signature_{}_{}".format(SUFFIX, TIMESTAMP)

SIGNATURE_OUT = f"{OUTPUT_PATH}/{sign_job_name}"

arguments = {"--input_data_path":INPUT_DATA_SOURCE_PATH,\
             "--signature_task": DATA_CONFIG,\
             "--output_bucket":BUCKET,\
             "--output_bucket_project": PROJECT,\
            "--output_bucket_path":SIGNATURE_OUT,\
             "--workflow_id":str(TIMESTAMP),\
             "--with_calendar_features": False, \
            "--with_power_transform": False \
            "--scaler_name": None, \
            "--moving_average_window_size": None, \
            "--cut_outliers_percentage": None
            }

signature_job = builder.job_file(f'{SCRIPT_PATH}/signature_lgbm.py')\
.job_id(sign_job_name)\
.py_file(f'{SCRIPT_PATH}/apigee_ingest_utils.py')\
.py_file(f'{SCRIPT_PATH}/ai4ops_db.py')\
.py_file(f'{SCRIPT_PATH}/yarn_logging.py')\
.file(f'{SCRIPT_PATH}/jobs/{DATA_CONFIG}')\
.arguments(**arguments)\
.build_job()



signature_executor = DataprocExecutor(signature_job, session)

In [ ]:
signature_res = signature_executor.submit_job(run_async=True)

In [ ]:
sleep(60)
state = signature_executor.get_job_state()

print('State : {}'.format(state))
if state not in ['DONE', 'RUNNING']:
    raise RuntimeError('Previous workflow step was failed')

In [ ]:
transition_signature = {
    "SIGNATURE_JOB_ID": sign_job_name,
    "SIGNATURE_TIMESTAMP": TIMESTAMP,
    "SIGNATURE_BUCKET": BUCKET,
    "SIGNATURE_TRAIN": f"{SIGNATURE_OUT}/LGBM-TRAIN-{TIMESTAMP}.csv",
    "SIGNATURE_VAL": f"{SIGNATURE_OUT}/LGBM-VAL-{TIMESTAMP}.csv",
    "SIGNATURE_TEST": f"{SIGNATURE_OUT}/LGBM-TEST-{TIMESTAMP}.csv",
    "SIGNATURE_SCALER": f"{SIGNATURE_OUT}/LGBM-SCL-{TIMESTAMP}.pkl",
    "SIGNATURE_DROP_KEYS": f"{SIGNATURE_OUT}/LGBM-DROP-KEYS-{TIMESTAMP}.txt",
    "SIGNATURE_STATE": state"
}

print(transition_signature)

with open('api_transition_signature.json', 'w') as file:
     file.write(json.dumps(transition_signature)) 

<a id="train"></a>
## Train LightGBM Model

See https://console.cloud.google.com/mlengine/jobs

[back to Table Of Contents](#tc)

In [ ]:
SCRIPT_PATH = "poc/models/gcp/lightgbm"
signature_transition = get_transition('api_transition_signature.json')
print(signature_transition)
state = signature_transition.get('SIGNATURE_STATE', '')
print('State: {}'.format(state))
if state not in ['DONE']:
    raise RuntimeError('Previous workflow step was failed')


SIGNATURE_TRAIN = transition.get('SIGNATURE_TRAIN', '')
SIGNATURE_VAL = transition.get('SIGNATURE_VAL', '')
SIGNATURE_TEST = transition.get('SIGNATURE_TEST', '')
SIGNATURE_SCALER = transition.get('SIGNATURE_SCALER', '')
CATEGORICAL_COLUMNS = ''
EXCLUDED_COLUMNS = AI_PLATFORM_TRAIN_EXCLUDED_COLUMNS

SCALE_TIER = 'STANDARD_1'




<a id="train_with_default"></a>
### Train With Default Parameters
[back to Table Of Contents](#tc)

In [ ]:
ALLOW_TUNING = False

In [ ]:
TIMESTAMP=int(datetime.now().timestamp())
TRAIN_JOB_NAME=f"api_ai4ops_lgbm_{TIMESTAMP}"
JOB_DIR=f"gs://{BUCKET}/apigee_history/apigee/models/lightgbm/{TRAIN_JOB_NAME}"
ERR_LOG_PATH_GS=f"apigee_history/apigee/models/lightgbm/{TRAIN_JOB_NAME}/output"
TRAINED_MODEL_PATH_GS = f"apigee_history/apigee/models/lightgbm/{TRAIN_JOB_NAME}/model"

training_input = {
  "scaleTier": SCALE_TIER,
  "masterConfig": {
    "imageUri": "gcr.io/kohls-kos-cicd/ai4ops_lgbm_image"
  },
  "region": AI_PLATFORM_REGION,
  "jobDir": JOB_DIR
}

args = {
    '--bucket_id': BUCKET, \
  '--train_data_path_gs': SIGNATURE_TRAIN, \
  '--val_data_path_gs': SIGNATURE_VAL, \
  '--train_val_data_folders_prefix_gs': "", \
  '--err_log_path_gs': ERR_LOG_PATH_GS, \
  '--trained_model_path_gs': TRAINED_MODEL_PATH_GS, \
  '--is_incremental_training': False, \
  '--boosting_type': "gbdt", \
  '--num_leaves': 742, \
  '--max_depth': 24, \
  '--learning_rate': 0.077748582911491323, \
  '--n_estimators': 153, \
  '--subsample_for_bin': 200000, \
  '--objective': "mse", \
  '--eval_metric': "mae", \
  '--obj_penalty': 1, \
  '--metrics': "l1,l2", \
  '--min_split_gain'"" 0.034919206912700895, \
  '--min_child_weight': 26.516502633376557, \
  '--min_child_samples': 1, \
  '--subsample': 0.89397221915183522, \
  '--subsample_freq': 38, \
  '--colsample_bytree': 0.53285653662681576, \
  '--reg_alpha': 34.255124175030247, \
  '--reg_lambda': 18.894649785140459, \
  '--n_jobs': -1, \
  '--early_stopping_rounds'; 10, \
  '--importance_type': "split", \
  '--categorical_feature': CATEGORICAL_COLUMNS, \
  '--target': "var1(t)", \
  '--excluded': EXCLUDED_COLUMNS
}


ai_train_job = AIJob(TRAIN_JOB_NAME, training_input)

ai_train_job.set_args(args)

In [ ]:
train_session = Session(BUCKET, AI_PLATFORM_REGION, CLUSTER, PROJECT)
defaulf_train_executor = AIPlatformJobExecutor(train_session, ai_train_job)

response=defaulf_train_executor.submit_train_job(30, 20)

state = response.state

In [ ]:
transition_train = {
    "TRAIN_JOB_ID": TRAIN_JOB_NAME,
    "TRAIN_JOB_DIR": JOB_DIR,
    "TRAIN_STATE": state,
    "TRAINED_MODEL": f"{JOB_DIR}/model"
}

print(transition_train)

with open('api_transition_train.json', 'w') as file:
     file.write(json.dumps(transition_train)) 

<a id="tuning"></a>
### Hyperparameters Tuning
[back to Table Of Contents](#tc)

In [ ]:
ALLOW_TUNING = True

In [ ]:
import pyyaml

TIMESTAMP=int(datetime.now().timestamp())
TUNING_JOB_NAME=f"api_ai4ops_tuning_lgbm_{TIMESTAMP}"
JOB_DIR=f"gs://{BUCKET}/apigee_history/apigee/models/lightgbm/{TUNING_JOB_NAME}"
ERR_LOG_PATH_GS=f"apigee_history/apigee/models/lightgbm/{TUNING_JOB_NAME}/output"
TRAINED_MODEL_PATH_GS = f"apigee_history/apigee/models/lightgbm/{TUNING_JOB_NAME}/model"

training_input = {
  "scaleTier": SCALE_TIER,
  "masterConfig": {
    "imageUri": "gcr.io/kohls-kos-cicd/ai4ops_lgbm_image"
  },
  "region": AI_PLATFORM_REGION,
  "jobDir": JOB_DIR
}

args = {
    '--bucket_id': BUCKET, \
    '--is_hyperparameters_tuning': True,\
  '--train_data_path_gs': SIGNATURE_TRAIN, \
  '--val_data_path_gs': SIGNATURE_VAL, \
  '--train_val_data_folders_prefix_gs': "", \
  '--err_log_path_gs': ERR_LOG_PATH_GS, \
  '--trained_model_path_gs': TRAINED_MODEL_PATH_GS, \
  '--is_incremental_training': False, \
  '--boosting_type': "gbdt", \
  '--num_leaves': 7, \
  '--learning_rate': 0.077748582911491323, \
  '--subsample_for_bin': 200000,\
  '--objective': "mse", \
  '--eval_metric': "mae", \
  '--obj_penalty': 1, \
  '--metrics': "l1,l2", \
  '--min_split_gain'"" 0.00021777905410443098, \
  '--min_child_weight': 15.299041042613257, \
  '--min_child_samples': 171, \
  '--subsample': 0.7424239139415899, \
  '--subsample_freq': 57, \
  '--colsample_bytree': 0.4449652059931909, \
  '--n_jobs': -1, \
  '--early_stopping_rounds'; 10, \
  '--importance_type': "split", \
  '--categorical_feature': CATEGORICAL_COLUMNS, \
  '--target': "var1(t)", \
  '--excluded': EXCLUDED_COLUMNS
}


ai_tuning_job = AIJob(TUNING_JOB_NAME, training_input)


ai_tuning_job.set_args(args)
ai_tuning_job.load_hyperparameters_from_file(TUNING_CONFIG_FILE)

In [ ]:
tuning_executor = AIPlatformJobExecutor(train_session, ai_tuning_job)

response = tuning_executor.submit_train_job(60, 60)
state = response.state

In [ ]:
transition_tuning = {
    "TRAIN_JOB_ID": TUNING_JOB_NAME,
    "TRAIN_JOB_DIR": JOB_DIR,
    "TRAIN_STATE": state,
    "TRAINED_MODEL": f"{JOB_DIR}/model"
}

print(transition_tuning)

with open('api_transition_tuning.json', 'w') as file:
     file.write(json.dumps(transition_tuning)) 

<a id="deployment"></a>
## Trained Model Deployment
See AI Platform Models https://console.cloud.google.com/mlengine/models
<br/>[back to Table Of Contents](#tc)

In [ ]:
SCRIPT_PATH = "poc/models/gcp/lightgbm"

In [ ]:
if ALLOW_TUNING:
    train_out = get_transition('api_transition_tuning.json')
else:
    train_out = get_transition('api_transition_train.json')

In [ ]:
state = train_out.get('TRAIN_STATE', '')

In [ ]:
TRAIN_JOB_ID = train_out.get('TRAIN_JOB_ID', '')
TRAIN_JOB_DIR = train_out.get('TRAIN_JOB_DIR', '')
TRAINED_MODEL = train_out.get('TRAINED_MODEL', '')

In [ ]:
if ALLOW_TUNING:
    OBJECTIVE_VALUE_IS_MAXIMUM_NEEDED = False
    PRINT_INFO = True
    best_trial = AIPlatformJobExecutor
    .get_best_hp_tuning_result(train_session._cloudml, PROJECT, TRAIN_JOB_ID, OBJECTIVE_VALUE_IS_MAXIMUM_NEEDED, PRINT_INFO)
     
    
    job_api.delete_path_from_gs(BUCKET, TRAINED_MODEL)
    
    TRAINED_MODEL=f"{TRAIN_JOB_DIR}/model_trial_{best_trial}"
    
    job_api.copy_folder_gs(BUCKET, TRAINED_MODEL, f"{TRAIN_JOB_DIR}/model")

In [ ]:
SCRIPT_PATH = "poc/models/gcp/lightgbm/ai_platform_predictions"
SIGNATURE_SCALER = signature_transition.get('SIGNATURE_SCALER', '')
SIGNATURE_DROP_KEYS = signature_transition.get('SIGNATURE_DROP_KEYS', '')
SIGNATURE_BUCKET = signature_transition.get('SIGNATURE_BUCKET', '')

SCALER_GCS_PATH = f'gs://{SIGNATURE_BUCKET}/{SIGNATURE_SCALER}'
DROP_KEYS_GCS_PATH = f'gs://{SIGNATURE_BUCKET}/{SIGNATURE_DROP_KEYS}'
DEPLOYMENT_PATH = f'gs://{SIGNATURE_BUCKET}/deployment'

MODEL_NAME = AI_PALTFORM_MODEL_NAME
VERSION_NAME = AI_PLATFORM_MODEL_BASE_VERSION




In [ ]:
import shutil

PREDICTOR_PACKAGE_NAME="custom_predictor"
PREDICTOR_PACKAGE_VERSION="0.1"
PREDICTOR_PACKAGE="${PREDICTOR_PACKAGE_NAME}-${PREDICTOR_PACKAGE_VERSION}.tar.gz"
STAGING_DIR = f'{TRAINED_MODEL}/staging'
DIST_DIR="dist"

script_args = ['sdist', '--formats=gztar', f'--dist-dir={DIST_DIR}'] 

job_api.create_package(f"{SCRIPT_PATH}/setup.py", )


job_api.upload_file_to_gs(PROJECT, BUCKET, f'{DIST_DIR}/{PREDICTOR_PACKAGE}', STAGING_DIR)


shutil.rmtree(DIST_DIR)
shutil.rmtree(f'{PREDICTOR_PACKAGE_NAME}.egg-info')


In [ ]:
deplpoy_job_input = {
  'pythonVersion': "3.5", \
  'deploymentUri': TRAINED_MODEL, \
  'packageUris': [f'{STAGING_DIR}/{PREDICTOR_PACKAGE}'], \
  'autoScaling':{'minNodes':1},
  'runtimeVersion': '1.13'
  'predictionClass': 'custom_predictor.LGBMPredictor'
}

In [ ]:
AIPlatformJobExecutor
.submit_deploy_model_job(train_session, MODEL_NAME, VERSION_NAME, deplpoy_job_input, 20, 5)

In [ ]:
job_api.copy_file_gs(BUCKET,SCALER_GCS_PATH, f"{DEPLOYMENT_PATH}/{MODEL_NAME}_{VERSION_NAME}/scaler.pkl")
job_api.copy_file_gs(BUCKET,DROP_KEYS_GCS_PATH, f"{DEPLOYMENT_PATH}/{MODEL_NAME}_{VERSION_NAME}/drop_keys.txt")

In [ ]:
transition_deployment = {
    "MODEL_NAME": MODEL_NAME,
    "VERSION_NAME": VERSION_NAME,
    "MODEL_DIR": TRAINED_MODEL",
    "STAGING_DIR": STAGING_DIR,
    "DEPLOYMENT": f"{DEPLOYMENT_PATH}/{MODEL_NAME}_{VERSION_NAME}",
    "SCALER": f"{DEPLOYMENT_PATH}/{MODEL_NAME}_{VERSION_NAME}/scaler.pkl",
    "DROP_KEYS": f"{DEPLOYMENT_PATH}/{MODEL_NAME}_{VERSION_NAME}/drop_keys.txt"
}

print(transition_deployment)

with open('api_transition_deployment.json', 'w') as file:
     file.write(json.dumps(transition_deployment)) 

<a id="prediction"></a>
## Batch Prediction with TEST Dataset
See Dataproc Jobs: https://console.cloud.google.com/dataproc/jobs 
<br/>[back to Table Of Contents](#tc)

In [ ]:
SCRIPT_PATH = "{}/models/gcp/lightgbm/ai_platform_predictions".format(PROJECT_BASE_PATH)
deployment = get_transition('api_transition_deployment.json')
AI_PALTFORM_MODEL_NAME = deployment.get('MODEL_NAME', '')
AI_PALTFORM_MODEL_VERSION = deployment.get('VERSION_NAME', '')



In [ ]:
TIMESTAMP=int(datetime.now().timestamp())
BASE_PATH="apigee_history/apigee/metrics/history"
OUTPUT_DATA_PATH=f"{BASE_PATH}/lgbm-batch-predicted/{TIMESTAMP}"
MAX_PARALLEL_REQUESTS=4

EXCLUDED_INPUT_COLUMNS="time,metric_val,var1(t)"
PREDICTED_COLUMN_NAME="predicted"
OUTPUT_COLUMNS_MAPPING="metric_val=metric,time,var1(t)=value"

PREDICT_JOB_ID=f"api_ai4ops_batch_prediction_lgbm_{TIMESTAMP}"


arguments = {"--project_id":PROJECT,\
             "--bucket_name": BUCKET,\
             "--model_name":AI_PALTFORM_MODEL_NAME,\
             "--version_name": AI_PALTFORM_MODEL_VERSION,\
            "--input_data_file":SIGNATURE_TEST,\
             "--output_data_path":OUTPUT_DATA_PATH,\
             "--excluded_input_columns": EXCLUDED_INPUT_COLUMNS, \
            "--predicted_column_name": PREDICTED_COLUMN_NAME \
            "--output_columns_mapping": OUTPUT_COLUMNS_MAPPING, \
            "--samples_count_in_chunk": 800, \
            "--max_parallel_requests": MAX_PARALLEL_REQUESTS,
             "--drop_by_nan_columns" : ""
            }

predict_job = builder.job_file(f'{SCRIPT_PATH}/batch_predictions.py')\
.job_id(PREDICT_JOB_ID)\
.arguments(**arguments)\
.build_job()

predict_executor = DataprocExecutor(predict_job, session)


In [ ]:
prediction_res = predict_executor.submit_job(run_async=True)

In [ ]:
sleep(60)
state = predict_executor.get_job_state()

print('State : {}'.format(state))
if state not in ['DONE', 'RUNNING']:
    raise RuntimeError('Previous workflow step was failed')

In [ ]:
transition_prediction = {
    "PREDICTION_JOB_ID": PREDICT_JOB_ID,
    "PREDICTION_TIMESTAMP": TIMESTAMP,
    "PREDICTION_BUCKET": BUCKET,
    "PREDICTION_OUTPUT": OUTPUT_DATA_PATH,
    "PREDICTION_STATE": state,
    "PREDICTION_EXCLUDED_INPUT_COLUMNS": EXCLUDED_INPUT_COLUMNS,
    "PREDICTION_PREDICTED_COLUMN_NAME": PREDICTED_COLUMN_NAME,
    "PREDICTION_OUTPUT_COLUMNS_MAPPING": OUTPUT_COLUMNS_MAPPING
}

print(transition_prediction)

with open('api_transition_prediction.json', 'w') as file:
     file.write(json.dumps(transition_prediction)) 

<a id="detection"></a>
## Anomalies Detection with TEST Dataset. Publish to Grafana
See Dataproc Jobs: https://console.cloud.google.com/dataproc/jobs 
<br/>[back to Table Of Contents](#tc)

In [ ]:
PROJECT_BASE_PATH="poc"
SCRIPT_PATH = "{}/spark/ingest".format(PROJECT_BASE_PATH)
prediction = get_transition('api_transition_prediction.json')

state = transition.get('PREDICTION_STATE', '')
print('State: {}'.format(state))
if state not in ['DONE']:
    raise RuntimeError('Previous workflow step was failed')
    
PREDICTION_BUCKET= prediction.get('PREDICTION_BUCKET', '')
PREDICTION_OUTPUT = prediction.get('PREDICTION_OUTPUT', 
                                   
THRESHOLD = 4.5
# USE_INVERSE_TRANSFORM = False

ANOMALIES_BASE_VERSION = 'LGBM Traffic Not 200 May Good' 
ANALYTICS_PATH = 'apigee_history/apigee/metrics/history/lgbm-analytics'
                                   
INVERSE_TRANSFORMER_PATH = (transition_deployment.get('SCALER', '') if USE_INVERSE_TRANSFORM 
                                          else '')
STAT_DEPLOYMENT_PATH = "deployment/{}_{}".format(transition_deployment.get('MODEL_NAME', ''), 
                                                               transition_deployment.get('VERSION_NAME', ''))
if not INVERSE_TRANSFORMER_PAT:
    STAT_FILE="stat.csv"
else
    STAT_FILE="inverse_stat.csv"                        


In [ ]:
TIMESTAMP=int(datetime.now().timestamp())

DETECTION_JOB_ID=f"api_ai4ops_anomaly_detection_{TIMESTAMP}"

VERSION=f"Ver.{TIMESTAMP}: {ANOMALIES_BASE_VERSION} THRE{THRESHOLD}"
DB_SECRET="kohls_db.txt"
RESOURCES="/opt/dataproc/.resources"
PREDICTION_PATH=f"gs://{PREDICTION_BUCKET}/{PREDICTION_OUTPUT}"
ANOMALY_NEIGHBORHOOD_SIZE = 1

arguments = {
            "--db_credentials_gcs_file_path" : f"gs://${BUCKET}/resources/${DB_SECRET}", \
            "--res_path" : RESOURCES, \
            "--predictions_path": PREDICTION_PATH, \
            "--inverse_transformer_name": job_api.get_file_name(INVERSE_TRANSFORMER_PATH),\
            "--anomaly_threshold":THRESHOLD,\
            "--anomaly_neighborhood_size": ANOMALY_NEIGHBORHOOD_SIZE, \
            "--version": VERSION \
            "--output_analytics_project": PROJECT, \
            "--output_analytics_bucket": BUCKET, \
            "--output_analytics_path": f"{ANALYTICS_PATH}/{TIMESTAMP}",
            "--output_stat_project" : PROJECT,\
            "--output_stat_bucket" : BUCKET,\
            "--output_stat_path" : STAT_DEPLOYMENT_PATH,\
            "--output_stat_file" : STAT_FILE
            }

detect_job = builder.job_file(f'{SCRIPT_PATH}/anomaly_detection.py')\
.job_id(DETECTION_JOB_ID)\
.py_file(f'{SCRIPT_PATH}/apigee_ingest_utils.py')\
.py_file(f'{SCRIPT_PATH}/ai4ops_db.py')\
.py_file(f'{SCRIPT_PATH}/yarn_logging.py')\
.py_file(f'{SCRIPT_PATH}/yarn_plotter.py')\
.file(INVERSE_TRANSFORMER_PATH)\
.jars(f"gs://{BUCKET}/resources/mysql-connector-java-8.0.16.jar")
.arguments(**arguments)\
.build_job()

detect_executor = DataprocExecutor(detect_job, session)

In [ ]:
detect_res = detect_executor.submit_job(run_async=True)

In [ ]:
sleep(60)
state = detect_executor.get_job_state()

print('State : {}'.format(state))
if state not in ['DONE', 'RUNNING']:
    raise RuntimeError('Previous workflow step was failed')

In [ ]:
transition_anomalies = {
    "ANOMALIES_JOB_ID": DETECTION_JOB_ID,
    "ANOMALIES_TIMESTAMP": TIMESTAMP,
    "ANOMALIES_ANALYTICS": BUCKET,
    "ANOMALIES_STATE": state,
    "ANOMALIES_VERSION": VERSION,
    "STAT_PATH": f"gs://{BUCKET}/{STAT_DEPLOYMENT_PATH}/${STAT_FILE}"
}

print(transition_prediction)

with open('api_transition_anomalies.json', 'w') as file:
     file.write(json.dumps(transition_anomalies)) 

In [ ]:
transition = get_transition('api_transition_anomalies.json')
print(transition)
state = transition.get('ANOMALIES_STATE', '')
print('State: {}'.format(state))
if state not in ['DONE']:
    raise RuntimeError('Previous workflow step was failed')

ANOMALIES_VERSION = transition.get('ANOMALIES_VERSION', '')

ANOMALIES_ANALYTICS = transition.get('ANOMALIES_ANALYTICS', '')
ANOMALIES_VERSION = ANOMALIES_VERSION
ANOMALIES_VERSION_ENC = ANOMALIES_VERSION.replace(' ', '%20')
GRAFANA_REF = ('<a id="grafana"></a><h2>Grafana Anomalies Dashboard</h2><a href="http://ai4ops-grafana-0:8080/d/1rJbPKnzWk1/ai4ops-versioned-anomalies?orgId=1&' + 
            'var-anomaly_metric_name={}&var-anomaly_version={}">{}</a>'.format(ANOMALIES_EXAMPLE_METRIC, 
                                                                           ANOMALIES_VERSION_ENC,
                                                                             ANOMALIES_VERSION))
from IPython.display import HTML, display
display(HTML(GRAFANA_REF))

<a id="analytics"></a>
## Analytics
[back to Table Of Contents](#tc)

In [ ]:
import os
shutil.rmtree('anomalies_analytics')
os.mkdir('anomalies_analytics')

job_api.download_folder_from_gs(BUCKET, ANOMALIES_ANALYTICS, 'anomalies_analytics')


In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('anomalies_analytics')

<a id="gcs"></a>
## Push Notebook to GCS Bucket
[back to Table Of Contents](#tc)

In [ ]:
from IPython.display import Javascript

script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''

def notebook_save():
    Javascript(script)
    print('This notebook has been saved')
notebook_save()

In [ ]:
!gsutil cp api_anomalies_batch_workflow.ipynb gs://ai4ops-main-storage-bucket/ai4ops-source/ai4ops-jupyter-ds-03/api/